## Create an S3 bucket to save participants' experiment data

In [1]:
import boto3

# Create S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Define bucket name
bucket_name = 'save-exp-data002'

# Create the S3 bucket
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f'Bucket {bucket_name} created successfully.')
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f'Bucket {bucket_name} already exists and is owned by you.')
except Exception as e:
    print(f'Error creating bucket: {e}')


Bucket save-exp-data002 created successfully.


## Create an S3 bucket to store experiment coding

In [2]:
import boto3
import os

# Create S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Define bucket name
bucket = 'final-project-exp003'

# Create S3 bucket
try:
    s3.create_bucket(Bucket=bucket)
    print(f'Bucket {bucket} created successfully.')
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f'Bucket {bucket} already exists and is owned by you.')
except Exception as e:
    print(f'Error creating bucket: {e}')

Bucket final-project-exp003 created successfully.


## Make the S3 bucket saving the experiment code public

In [3]:
import boto3

# Create S3 client
s3 = boto3.client('s3', region_name='us-east-1')


# Update bucket public access settings to allow public access
try:
    s3.put_public_access_block(
        Bucket=bucket,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': False,
            'IgnorePublicAcls': False,
            'BlockPublicPolicy': False,
            'RestrictPublicBuckets': False
        }
    )
    print(f'Public access settings for bucket {bucket} updated successfully.')
except Exception as e:
    print(f'Error updating public access settings for bucket: {e}')


Public access settings for bucket final-project-exp003 updated successfully.


## Ensure the object in the S3 bucket is also accessible and upload the experiment coding to the bucket.

In [4]:
import boto3
import os
import json

# Create S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Define bucket name
bucket = 'final-project-exp003'

# Define bucket policy
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AddPerm",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucket}/*"
        }
    ]
}

# Convert the policy to a JSON string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy on the given bucket
try:
    s3.put_bucket_policy(Bucket=bucket, Policy=bucket_policy)
    print(f'Bucket policy for {bucket} set successfully.')
except Exception as e:
    print(f'Error setting bucket policy: {e}')

# Define the file name and path
file_name = 'dual_coding_exp.zip'
file_path = os.path.join(os.getcwd(), file_name)

# Upload the file to the S3 bucket
try:
    s3.upload_file(file_path, bucket, file_name)
    print(f'File {file_name} uploaded successfully to {bucket}.')
except Exception as e:
    print(f'Error uploading file: {e}')


Bucket policy for final-project-exp003 set successfully.
File dual_coding_exp.zip uploaded successfully to final-project-exp003.


## Create an EC2 instance to run the experiment

In [14]:
import boto3

# Create EC2 client
ec2 = boto3.client('ec2', region_name='us-east-1')

# Get default VPC ID
def get_default_vpc_id():
    response = ec2.describe_vpcs(Filters=[{'Name': 'isDefault', 'Values': ['true']}])
    if response['Vpcs']:
        return response['Vpcs'][0]['VpcId']
    else:
        raise Exception('No default VPC found')

# Get the default VPC ID
try:
    default_vpc_id = get_default_vpc_id()
    print(f'Default VPC ID: {default_vpc_id}')
except Exception as e:
    print(f'Error getting default VPC ID: {e}')
    default_vpc_id = None  # Handle this appropriately

# Get the latest Amazon Linux 2 AMI ID
def get_latest_amazon_linux_ami():
    response = ec2.describe_images(
        Owners=['amazon'],
        Filters=[
            {'Name': 'name', 'Values': ['amzn2-ami-hvm-*-x86_64-gp2']},
            {'Name': 'state', 'Values': ['available']}
        ]
    )
    images = sorted(response['Images'], key=lambda x: x['CreationDate'], reverse=True)
    if images:
        return images[0]['ImageId']
    else:
        raise Exception('No Amazon Linux 2 AMI found')

# Get the latest Amazon Linux 2 AMI ID
try:
    latest_ami_id = get_latest_amazon_linux_ami()
    print(f'Latest Amazon Linux 2 AMI ID: {latest_ami_id}')
except Exception as e:
    print(f'Error getting latest Amazon Linux 2 AMI ID: {e}')
    latest_ami_id = None  # Handle this appropriately

# Check if security group already exists
def get_security_group_id(security_group_name, vpc_id):
    response = ec2.describe_security_groups(Filters=[
        {'Name': 'group-name', 'Values': [security_group_name]},
        {'Name': 'vpc-id', 'Values': [vpc_id]}
    ])
    if response['SecurityGroups']:
        return response['SecurityGroups'][0]['GroupId']
    return None

# Define security group parameters
security_group_name = 'my-security-group'
security_group_description = 'Security group for allowing SSH and HTTP access'

# Create or get security group ID
try:
    security_group_id = get_security_group_id(security_group_name, default_vpc_id)
    if not security_group_id:
        response = ec2.create_security_group(
            GroupName=security_group_name,
            Description=security_group_description,
            VpcId=default_vpc_id
        )
        security_group_id = response['GroupId']
        print(f'Security Group Created {security_group_id} in VPC {default_vpc_id}.')

        # Add a rule to allow SSH traffic from anywhere and HTTP traffic from the internet
        ec2.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': 22,
                 'ToPort': 22,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                {'IpProtocol': 'tcp',
                 'FromPort': 80,
                 'ToPort': 80,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
        )
        print(f'Ingress Successfully Set for Security Group {security_group_name}')
    else:
        print(f'Security Group {security_group_name} already exists with ID {security_group_id}.')
except Exception as e:
    print(f'Error creating or getting security group: {e}')
    security_group_id = None  # Handle this appropriately

# Launch an EC2 instance
try:
    if security_group_id:
        instances = ec2.run_instances(
            ImageId=latest_ami_id,  # Use the obtained latest AMI ID
            MinCount=1,
            MaxCount=1,
            InstanceType='t2.micro',
            KeyName='vockey',
            SecurityGroupIds=[security_group_id],
        )
        instance_id = instances['Instances'][0]['InstanceId']
        print(f'EC2 Instance {instance_id} launched successfully.')
    else:
        print('Security group ID is invalid. Instance launch aborted.')
except Exception as e:
    print(f'Error launching EC2 instance: {e}')


Default VPC ID: vpc-0d524f8d985955be3
Latest Amazon Linux 2 AMI ID: ami-0babef7f814da8951
Security Group my-security-group already exists with ID sg-0b917930336fe7475.
EC2 Instance i-04a03d48345d89929 launched successfully.


## Create a RDS instance to recieve the experiment data when the participants finish the experiment

In [15]:
import boto3
import pymysql
import time

# Create RDS client
rds = boto3.client('rds', region_name='us-east-1')

# Create RDS instance
response = rds.create_db_instance(
    DBInstanceIdentifier='final-project-database001',
    DBName='finaldata001',
    MasterUsername='username',
    MasterUserPassword='password',
    DBInstanceClass='db.t3.micro',
    Engine='mysql',
    AllocatedStorage=5
)

# Wait for the RDS instance to be available
print("Creating RDS instance. This may take a few minutes...")
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier='final-project-database001')
print("RDS instance is now available.")

# Get RDS instance connection information
db = rds.describe_db_instances(DBInstanceIdentifier='final-project-database001')['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(f"Endpoint: {ENDPOINT}")
print(f"Port: {PORT}")
print(f"DBInstanceIdentifier: {DBID}")

# Create the database and tables
# Wait a few minutes to ensure the RDS instance is fully ready
time.sleep(300)


Creating RDS instance. This may take a few minutes...
RDS instance is now available.
Endpoint: final-project-database001.cv7jvzss5xbo.us-east-1.rds.amazonaws.com
Port: 3306
DBInstanceIdentifier: final-project-database001


In [16]:
import boto3

# Define the endpoint and port
ENDPOINT = 'final-project-database001.cv7jvzss5xbo.us-east-1.rds.amazonaws.com'
PORT = 3306

## Grant the EC2 instance permission to access the RDS instance so the experiment data could be saved successfully

In [17]:
import boto3

# Get the security group ID for the RDS instance
rds = boto3.client('rds', region_name='us-east-1')
db = rds.describe_db_instances(DBInstanceIdentifier='final-project-database001')['DBInstances'][0]
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust permissions for the security group to allow access on port 3306
ec2 = boto3.client('ec2')
try:
    ec2.authorize_security_group_ingress(
        GroupId=SGNAME,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': PORT,
             'ToPort': PORT,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
        ]
    )
    print("Permissions successfully adjusted.")
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)


Permissions already adjusted.


## Create a few tables to save different sections of the experiment data

In [26]:
# Connect to the RDS instance
try:
    connection = pymysql.connect(
        host='final-project-database001.cv7jvzss5xbo.us-east-1.rds.amazonaws.com',
        user='username',
        password='password',
        port=PORT
    )
    print("Connected to the RDS instance successfully.")
    
    try:
        with connection.cursor() as cursor:
            # Create database
            cursor.execute("CREATE DATABASE IF NOT EXISTS finaldata001")
            cursor.execute("USE finaldata001")
            
            # Create tables
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS EmailAddressInput (
                EmailAddress VARCHAR(255) UNIQUE,
                FutureContactConsent BOOLEAN
            )
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS Demographics (
                EmailAddress VARCHAR(255) UNIQUE,
                Age INT,
                Sex VARCHAR(255),
                Ethnicity VARCHAR(255),
                Race VARCHAR(255)
            )
            """)
            connection.commit()
        print("Database and tables created successfully.")
    finally:
        connection.close()
except pymysql.MySQLError as e:
    print(f"Error connecting to RDS: {e}")

Connected to the RDS instance successfully.
Database and tables created successfully.


In [27]:
import pymysql

# Connect to the RDS instance
try:
    connection = pymysql.connect(
        host='final-project-database001.cv7jvzss5xbo.us-east-1.rds.amazonaws.com',
        user='username',
        password='password',
        database='finaldata001',
        port=3306
    )
    print("Connected to the RDS instance successfully.")
    
    try:
        with connection.cursor() as cursor:
            # Create or alter the Demographics table to add VVIQ columns
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS VVIQResponses (
                EmailAddress VARCHAR(255) UNIQUE,
                VVIQ_Q1 INT,
                VVIQ_Q2 INT,
                VVIQ_Q3 INT,
                VVIQ_Q4 INT,
                VVIQ_Q5 INT,
                VVIQ_Q6 INT,
                VVIQ_Q7 INT,
                VVIQ_Q8 INT,
                VVIQ_Q9 INT,
                VVIQ_Q10 INT,
                VVIQ_Q11 INT,
                VVIQ_Q12 INT,
                VVIQ_Q13 INT,
                VVIQ_Q14 INT,
                VVIQ_Q15 INT,
                VVIQ_Q16 INT
            )
            """)
            connection.commit()
        print("Table created or altered successfully.")
    finally:
        connection.close()
except pymysql.MySQLError as e:
    print(f"Error connecting to RDS: {e}")


Connected to the RDS instance successfully.
Table created or altered successfully.


In [28]:
import pymysql

# Connect to the RDS instance
try:
    connection = pymysql.connect(
        host='final-project-database001.cv7jvzss5xbo.us-east-1.rds.amazonaws.com',
        user='username',
        password='password',
        database='finaldata001',
        port=3306
    )
    print("Connected to the RDS instance successfully.")
    
    try:
        with connection.cursor() as cursor:
            # Create the AttentionCheck table
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS AttentionCheck (
                EmailAddress VARCHAR(255) UNIQUE,
                IdealParticipation BOOLEAN,
                AttentionCheckTask VARCHAR(255)
            )
            """)
            connection.commit()
        print("Table created or altered successfully.")
    finally:
        connection.close()
except pymysql.MySQLError as e:
    print(f"Error connecting to RDS: {e}")


Connected to the RDS instance successfully.
Table created or altered successfully.


## Use the code below to renew the experiment code in the S3 bucket if the experiment code has been updated

In [4]:
import boto3
import os
import json

# Create S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Define bucket name
bucket = 'final-project-exp003'

# Define the file name and path
file_name = 'dual_coding_exp.zip'
file_path = os.path.join(os.getcwd(), file_name)

# Upload the file to the S3 bucket, replacing the existing file
try:
    s3.upload_file(file_path, bucket, file_name)
    print(f'File {file_name} uploaded successfully to {bucket}, replacing the existing file.')
except Exception as e:
    print(f'Error uploading file: {e}')


File dual_coding_exp.zip uploaded successfully to final-project-exp003, replacing the existing file.
